# Using python to interface with BTLS

1.    Define the inputs as python objects.
2.    Write the inputs to files for BTLS to read.
3.    Configure the simulation.
4.    Execute `PyBTLS.run()`.
5.    Analysis of outputs.

The necessary imports are called as follows:

In [1]:
import PyBTLS as btls
from PyBTLS.py.traffic.PyBtlsDefaultTraffic import Auxerre #OLD CODE, BUT NO REPLACEMENT YET
import numpy as np

## 1. Define Inputs

### Define Bridges

Define the bridge using syntax similar to PyCBA.

In [2]:
L = [50.0] #m
R = [-1, 0, -1, 0] #Simply supported
bridge = btls.PyBridge(L=L, R=R, n_lane=1)

### Define Load Effects

Adding the influence lines corresponding to the load effects of interest can be done in two ways:

1. The first way is to use BTLS built-in influence line; e.g., this is Influence Line 1, which computes moment at midspan. See BTLS manual for full list of built-in load effects

In [3]:
bridge.add_load_effect(load_effect = 1) 

2. The second method to define load effects is by specifiying:
- load_effect ('M', 'V', or 'R' for moment, shear or reaction)
- poi         (The point where the load effect is to be measured)
- max_error   (The maximum error of the IL calculation. Error is relative to actual values.)

Some other available arguments are:
- lane_factor (The lane_factor. Can be a list for different factors for each lane, or a singular value for same lane_factor for all. default 1.0)
- POT         (Minimum LE to be included in Peak Over Threshold (POT) analysis. default to 0.0)

In [4]:
bridge.add_load_effect(load_effect = "M", poi = 3/4 * L[0], max_error = 0.001)
bridge.add_load_effect(load_effect = "M", poi = 1/4 * L[0], max_error = 0.001)
bridge.add_load_effect(load_effect = "V", poi = 1/4 * L[0], max_error = 0.001)

### Define Lane Flow
Lane flow can be constructed using:
  - **data frame** (NOT recommended as it is prone to error): Construction via data frame is done using the Pandas.DataFrame arguments and syntax. See Pandas documentation for details.
  - **csv file**: Construction via csv file requires the path to the csv file as a string argument: `path`        
  - **sample**: Construction via the provided samples can be done using the `load_default()` method by providing the sample number, either 1 or 2 (default 1)

In [5]:
lane_1 = btls.LaneFlow()
lane_1.load_default()

### Define the Bridge Lanes

The constructed lane flow object(s) can be added to a bridge to define the bridge lanes: a `BridgeFlow` object, which describes the flow of all the lanes in the bridge. Lane can be added at initialisation as `list` or `BtlsFlow` object, or after initialisation via:
  - `add()` method or other aliases of the method: `add_lane_flow()`, `append()`, `extend()`)
  - `insert(idx, lane_flow)` : insert new lane flow into the array
  - `list` indexing          : This is similar to regular list indexing, e.g. `bridge_flow[1] = lane_2`, `del bridge_flow[0]`, etc.

In [6]:
bridge_flow = btls.BridgeFlow(lane_1)

### Import traffic files

The files corresponding to the vehicle model are specified. These are currently:
- **Grave Model**: based on the 1980s French sites traffic data upon which the Eurocode was developed. See Caprani (2005) PhD Thesis for more. A specific site is specified.
- **Garage Model**: a kernel-density estimation bootstrapping of a measured traffic file.
- **Nominal Model**: a simple model of a nominal vehicle configuration with kernel-density perturbations

In [7]:
#nominal_vehicle = btls.NominalVehicle()
#nominal_vehicle.load_default()

#garage = btls.Garage()
#garage.load_default()

#kernels = btls.Kernels()
#kernels.load_default()

grave_traffic = Auxerre #OLD CODE, BUT NO REPLACEMENT YET

# 2. Write Inputs to File

Execute `write_as_txt()` or `write_as_csv()` for the files that BTLS needs:
- Bridges
- Influence lines
- Traffic file(s)
- Bridge flows

In [8]:
bridge_path = "Input/bridge.txt"
bridge_flow_path = "Input/lane_flow.csv"
il_path = "Input/IL.txt"
#nominal_vehicle_path = "Input/nominal_vehicle.csv"
#garage_path = "Input/garage.txt"
#kernels_path = "Input/kernels.csv"
grave_traffic_path = "Input/Traffic"

bridge.write_as_txt(il_path = il_path, bridge_path = bridge_path)
#nominal_vehicle.write_as_csv(path = nominal_vehicle_path)
#garage.write_as_txt(path = garage_path)
#kernels.write_as_csv(path = kernels_path)
grave_traffic.write_as_csv(path = grave_traffic_path)

bridge_flow._check_lane_clashes()
bridge_flow._check_lane_complete()

#Check that the number of lanes in the bridge_flow is equal to the number of lanes in the bridge
if not len(bridge_flow.lane_flow) == bridge.n_lane:
    ValueError("ERROR: Number of lanes in BridgeFlow is not equal to the number of lanes in Bridge")
bridge_flow.write_as_csv(path = bridge_flow_path)

# 3. Configuration

Firstly, set up `PyBTLS.ConfigData()` to point to the created files. 

Secondly, specificy further options for how the simulation is configured. See the documentation for more details.

Thirdly, specific the outputs required from BTLS.

In [9]:
config = btls.ConfigData()

config.veh_gen_grave(
        lanes_file=bridge_flow_path,
        traffic_folder=grave_traffic_path,
        no_days=10,
        truck_track_width=190.0,
        lane_eccentricity_std=0.0,
    )

"""
config.veh_gen_nominal(
    lanes_file=bridge_flow_path, 
    nominal_file=nominal_vehicle_path, 
    no_days=10, 
    lane_eccentricity_std=0.0)
"""

"""
config.veh_gen_garage(
    lanes_file=bridge_flow_path, 
    garage_file=garage_path, 
    file_format=4, 
    kernel_file="kernels_path", 
    no_days=10, 
    lane_eccentricity_std=20)
"""

config.flow_gen_NHM(
    vehicle_classification=1, 
    traffic_folder=grave_traffic_path)

# config.flow_gen_nominal(vehicle_classification=1, nominal_speed=36, nominal_gap=10)

# config.flow_gen_congestion(vehicle_classification=1, congested_spacing=26.1, congested_speed=49.7, congested_gap_coef_var=0.05)

# config.flow_gen_freeflow(vehicle_classification=1)

config.simulation(
    bridge_file=bridge_path,
    infline_file=il_path,
    infsurf_file="./IS.txt",
    calc_time_step=0.1,
    min_gvw=35,
)

# config.output_general()


config.output_BM(
    write_blockmax=True, 
    write_summary=True,
    write_vehicle=True,
    write_mixed=True,
    block_size_days=int(1)
    )

# config.output_POT(write_pot=True)

config.output_vehicle_file(
    write_vehicle_file=True,
    vehicle_file_format=4,
    write_flow_stats=True,
    buffer_size=100000)

config.output_stats(
    write_stats=True,
    write_cumulative=True,
    write_intervals=True,
    interval_size=int(24*3600)
    )

# config.output_fatigue(do_rainflow=True)

# 4. Execute BTLS

This is as simple as calling:

In [10]:
btls.run()

Reading influence line file: Input/IL.txt
Starting simulation...
Day complete...
Reading influence surface file: ./IS.txt


*** Warning: Influence Surface file could not be opened



Flushing buffer of 100000 vehicles at 10/1/0 19:6:18.6198
	1	2	3	4	5	6	7	8	9	10

Flushing buffer of 2323 vehicles at 11/1/0 0:0:8.02185


# 5. Analysis of Outputs

Call `read_outputs()` to read in the outputs for further processing. The output of the analysis is store in `btls.output` object.

## Vehicle File

The created vehicle file can be examined in a dataframe format, which is very useful for summary statistics and further data analysis.

In [11]:
btls.PyVehicle(path="Vehicles.txt", file_format="MON")

,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,1,1,2010,0,0,1.700,2,0,5302.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,1,1,2010,0,0,9.810,5,0,44516.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1001,1,1,2010,0,0,12.130,4,0,35967.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1001,1,1,2010,0,0,20.929,2,0,26952.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1001,1,1,2010,0,0,41.906,4,0,19979.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102318,1001,10,1,2010,23,59,31.627,2,0,14486.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102319,1001,10,1,2010,23,59,39.944,2,0,18873.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102320,1001,10,1,2010,23,59,54.224,3,0,30627.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102321,1001,10,1,2010,23,59,56.510,3,0,22923.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Loading Events Summary

A summary of the load effect outputs is available, for example, the number of events of each type in each block (for block maximum output).

In [12]:
bm_summary = btls.read_block_maxima_summary_file(froot="./", bridge_length=np.sum(bridge.L))
for i in bm_summary:
    display(i.head())

,index,event_1_truck,event_2_truck,event_3_truck
0,1,6911.1,7897.8,572.7
1,2,7478.0,6959.7,5829.7
2,3,6857.3,6715.9,4445.7
3,4,7213.8,7069.8,5169.8
4,5,6994.9,7850.9,4217.7


,index,event_1_truck,event_2_truck,event_3_truck
0,1,5406.3,6241.8,286.3
1,2,5780.1,5209.6,3693.2
2,3,5280.2,5584.7,3227.7
3,4,5481.0,5685.0,3322.4
4,5,5436.7,6113.7,2880.8


,index,event_1_truck,event_2_truck,event_3_truck
0,1,422.1,455.4,22.9
1,2,447.0,390.0,295.5
2,3,406.6,399.7,258.2
3,4,435.3,410.1,265.8
4,5,410.9,424.8,230.5


,index,event_1_truck,event_2_truck,event_3_truck
0,1,5108.5,6031.3,682.4
1,2,5635.5,5543.3,4108.5
2,3,5131.1,5586.1,2830.7
3,4,5414.2,5443.5,2892.2
4,5,5233.1,6113.2,2908.0


## Cumulative Statistics

Overall statistical information on each load effect.

In [13]:
btls.read_cumulative_statistics_file(froot = "./", bridge_length=np.sum(bridge.L))

,load_effect,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,109567,116996,116996,6.47,8020.46,3432.34,1530.42,2342200.17,-0.37,-0.86,102157.0,7391.0,19.0
1,2,109567,116996,116996,9.71,6267.87,2602.76,1148.38,1318773.68,-0.38,-0.88,102157.0,7391.0,19.0
2,3,109567,116996,116996,3.24,6526.06,2642.42,1186.71,1408291.89,-0.35,-0.89,102157.0,7391.0,19.0
3,4,109567,116996,116996,-70.37,480.68,200.84,92.70,8594.00,-0.36,-0.85,102157.0,7391.0,19.0


## Interval Statistics

Statistics of load effects within each interval. Useful for nonstationary flows.

In [14]:
interval_statistics = btls.read_interval_statistics_files(froot = "./", bridge_length=np.sum(bridge.L))
for i in interval_statistics:
    display(i.head())

,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11011,11765,11765,9.48,6241.77,2636.05,1181.06,1394893.71,-0.34,-0.86,10258.0,752.0,1.0
1,2,172800.0,11007,11782,11782,7.09,5780.13,2652.70,1187.60,1410387.72,-0.36,-0.89,10236.0,767.0,4.0
2,3,259200.0,10779,11460,11460,4.01,5584.65,2647.29,1175.27,1381250.70,-0.37,-0.87,10099.0,679.0,1.0
3,4,345600.0,10905,11602,11602,3.24,5685.03,2649.57,1188.18,1411767.69,-0.37,-0.88,10209.0,695.0,1.0
4,5,432000.0,10918,11655,11655,7.45,6113.73,2642.40,1190.62,1417581.92,-0.34,-0.90,10183.0,733.0,2.0


,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11011,11765,11765,-64.64,455.39,200.26,92.19,8498.39,-0.34,-0.85,10258.0,752.0,1.0
1,2,172800.0,11007,11782,11782,-61.48,446.98,201.48,92.91,8632.84,-0.37,-0.84,10236.0,767.0,4.0
2,3,259200.0,10779,11460,11460,-59.07,406.62,201.48,91.80,8426.80,-0.38,-0.83,10099.0,679.0,1.0
3,4,345600.0,10905,11602,11602,-62.16,435.32,201.58,92.73,8598.52,-0.37,-0.84,10209.0,695.0,1.0
4,5,432000.0,10918,11655,11655,-58.34,424.81,200.62,93.13,8672.73,-0.35,-0.86,10183.0,733.0,2.0


,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11011,11765,11765,28.43,6031.27,2594.66,1144.20,1309201.30,-0.37,-0.87,10258.0,752.0,1.0
1,2,172800.0,11007,11782,11782,21.28,5635.47,2611.96,1147.23,1316127.08,-0.39,-0.88,10236.0,767.0,4.0
2,3,259200.0,10779,11460,11460,12.03,5586.06,2607.34,1138.66,1296555.26,-0.40,-0.86,10099.0,679.0,1.0
3,4,345600.0,10905,11602,11602,9.71,5443.47,2610.12,1148.08,1318094.73,-0.40,-0.86,10209.0,695.0,1.0
4,5,432000.0,10918,11655,11655,22.36,6113.18,2601.37,1153.66,1330924.04,-0.37,-0.90,10183.0,733.0,2.0


,id,time,num_events,num_event_vehicles,num_event_trucks,min,max,mean,std_dev,variance,skeweness,kurtosis,event_1_truck,event_2_truck,event_3_truck
0,1,86400.0,11011,11765,11765,18.95,7897.76,3422.10,1523.58,2321292.35,-0.36,-0.85,10258.0,752.0,1.0
1,2,172800.0,11007,11782,11782,14.19,7478.04,3443.41,1531.22,2344628.36,-0.38,-0.87,10236.0,767.0,4.0
2,3,259200.0,10779,11460,11460,8.02,6857.26,3439.02,1517.08,2301524.61,-0.38,-0.84,10099.0,679.0,1.0
3,4,345600.0,10905,11602,11602,6.47,7213.81,3442.43,1532.51,2348598.17,-0.39,-0.85,10209.0,695.0,1.0
4,5,432000.0,10918,11655,11655,14.91,7850.92,3430.06,1536.57,2361053.55,-0.36,-0.89,10183.0,733.0,2.0


## Block Maxima - Mixed Event Types

The block maximum results without separating the events by the number of constituent vehicles.

In [15]:
blockmax_mm = btls.read_block_maxima_event_mixed_vehicles_file(froot="./", bridge_length=np.sum(bridge.L), file_format="MON")
blockmax_mm.head()

,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,7897.8,82302.1,45.06,2,2 vehicle(s) Vehicle object at 0x7fe2337941f0
1,1,2,6031.3,82302.2,47.31,2,2 vehicle(s) Vehicle object at 0x7fe2337942b0
2,1,3,6241.8,82302.0,42.80,2,2 vehicle(s) Vehicle object at 0x7fe233794340
3,1,4,455.4,82302.1,45.06,2,2 vehicle(s) Vehicle object at 0x7fe233794400
4,2,1,7478.0,92949.3,31.56,1,1 vehicle(s) Vehicle object at 0x7fe2337944c0


## Block Maxima - Separated Event Types

For the correct extrapolation, the events must be separated by the contributing number of vehicles (see Caprani (2005)).

In [16]:
bm_event = btls.read_block_maxima_event_separated_vehicles_file(froot="./", bridge_length=np.sum(bridge.L), file_format="MON")
for i in bm_event:
    display(i.head())

,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,7897.8,82302.1,45.06,2,2 vehicle(s) Vehicle object at 0x7fe232e55c70
1,1,2,6031.3,82302.2,47.31,2,2 vehicle(s) Vehicle object at 0x7fe232e55d30
2,1,3,6241.8,82302.0,42.80,2,2 vehicle(s) Vehicle object at 0x7fe232e55dc0
3,1,4,455.4,82302.1,45.06,2,2 vehicle(s) Vehicle object at 0x7fe232e55e80
4,2,1,6959.7,135432.0,36.17,2,2 vehicle(s) Vehicle object at 0x7fe232e55f40


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,572.7,78663.5,58.71,3,3 vehicle(s) Vehicle object at 0x7fe2337d54c0
1,1,2,682.4,78663.5,58.71,3,3 vehicle(s) Vehicle object at 0x7fe2337d5580
2,1,3,286.3,78663.5,58.71,3,3 vehicle(s) Vehicle object at 0x7fe2337d5610
3,1,4,22.9,78663.5,58.71,3,3 vehicle(s) Vehicle object at 0x7fe2337d56d0
4,2,1,5829.7,122907.1,50.92,3,3 vehicle(s) Vehicle object at 0x7fe2337d5790


,day,load_effect_num,max_load_effect,time_of_max,dist_to_datum,num_trucks,vehicles
0,1,1,6911.1,57141.3,33.95,1,1 vehicle(s) Vehicle object at 0x7fe23381b790
1,1,2,5108.5,57141.6,41.78,1,1 vehicle(s) Vehicle object at 0x7fe23381b850
2,1,3,5406.3,57140.9,23.50,1,1 vehicle(s) Vehicle object at 0x7fe23381b8e0
3,1,4,422.1,42116.3,24.50,1,1 vehicle(s) Vehicle object at 0x7fe23381b9a0
4,2,1,7478.0,92949.3,31.56,1,1 vehicle(s) Vehicle object at 0x7fe23381ba60


## Specific Event
Here we examine the critical 3 vehicle crossing event causing the extreme load effect for Load Effect 1. We examine their headway at the maximum loading event. We can use regular pandas expressions and methods.

In [17]:
# Shorthand for the output object we're interested in
df = bm_event[0]

# Filter only load_effect 1
df[df['load_effect_num'] == 1]

# Get the critical vehicles
critical_platoon = df["vehicles"][np.argmax(df.max_load_effect)]

#Calculate their headways
arrival_time_in_second = (critical_platoon.second + critical_platoon.minute * 60 + critical_platoon.hour * 3600).to_numpy()
headway = arrival_time_in_second[1:] - arrival_time_in_second[0:-1]

# Display the results
print("Headways of the critical vehicles for 3 vehicles crossing event are:", headway, "seconds")
display(critical_platoon)

Headways of the critical vehicles for 3 vehicles crossing event are: [0.917] seconds


,head,day,month,year,hour,minute,second,num_axle,num_axle_groups,gvw,...,AW11,AS11,AW12,AS12,AW13,AS13,AW14,AS14,AW15,AS15
0,1001,6,1,2010,16,4,13.419,5,0,53477.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,6,1,2010,16,4,14.336,5,0,56724.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
